In [88]:
import datetime as dt
import tzlocal
import pandas as pd
from icalendar import Calendar, Event



removeStrings = ["Abfuhrtermin"]    #strings to remove from event-summary

icsFiles = ["../personal.ics","../birthdays.ics"]

loctz = tzlocal.get_localzone() #local timezone

df = pd.DataFrame()

for file in icsFiles:


    g = open(file,'rb')
    gcal = Calendar.from_ical(g.read())
    for component in gcal.walk():
        if component.name == "VEVENT":
            
            start = component.decoded('dtstart')            

            isOnlyDate = type(start) == dt.date #some elements have only a date which reflects allDay-dates
            if isOnlyDate:
                start = dt.datetime(start.year,start.month,start.day,tzinfo=loctz)
            
            isYearly = False
            if component.has_key('RRULE'):
                rule = component.decoded('RRULE')
                isYearly = "YEARLY" in rule['FREQ'][0] == "YEARLY"
                if isYearly:
                    start = dt.datetime(dt.datetime.now().year,start.month,start.day,tzinfo=loctz)
                
            if start > dt.datetime.now(loctz):
                eventSummary = component.get('summary')
                for removeString in removeStrings:
                    eventSummary = eventSummary.replace(removeString,"")

                isYearly = True

                df = df.append({"Event":eventSummary, "Date":str(start.date()), "AllDay":isOnlyDate, "IsYearly":isYearly},ignore_index=True)

    g.close()

df.to_json("../personal.json",orient="records")
df

,Event,Date,AllDay,IsYearly
0,Imagine Dragons,2022-06-14,0.0,1.0
1,Michael Schlossbauer Geburtstag,2022-09-10,1.0,1.0
2,Grundsteuererklärung,2022-07-02,1.0,1.0
3,Posteo aufladen,2022-12-17,1.0,1.0
4,Waschmaschine warten?,2022-11-01,1.0,1.0
...,...,...,...,...
74,Marcel Amrhein (*2014),2022-06-30,1.0,1.0
75,Jens Väth (*1981),2022-11-13,1.0,1.0
76,Tanja Schloth (*1975),2022-08-19,1.0,1.0
77,🎂 Steffen Brehm (1988),2022-12-26,1.0,1.0
